In [84]:
import json_lines as jl
import nltk
import xmltodict
from ispras import texterra as isp
import pymorphy2 as pm2
import gensim
import time as tm

isp_api = isp.API('dd58e71c2c65fa4af14fb557f904ede7f3b30575')
#nltk.download()

In [48]:
comments = ''
with open('learn_scrapy\pda.jl', 'rb') as f:
    for item in jl.reader(f):
        comments = comments + item['comment']       #создаём один string на все комменты
                                                    #скорее всего, будет логичнее для N-граммов каждое предложение
                                                    #запоминать в один string и хранить как list()

In [49]:
#тут токенизируем комментарии
#есть подозрение, что я создаю слишком много сущностей 

tknzr = nltk.TweetTokenizer()
tokens = tknzr.tokenize(comments)
text = nltk.Text(tokens)
sents = nltk.sent_tokenize(comments)                         #строить n-граммную модель, возможно, буду 
                                                             #анализируя предложения по-отдельности
words = sorted([w.lower() for w in tokens if w.isalpha()])   #впоследствии буду отсекать наименее наиболее частотные слова

In [20]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

print(lexical_diversity(text))

0.10600020178435018


In [21]:
sent_tokens = list()                            #здесь храним предложения
for i in range(len(sents)):
    sent_tokens.append(tknzr.tokenize(sents[i]))    

In [22]:
#сох нормальные формы слов
morph = pm2.MorphAnalyzer()     
vocab_normalized = sorted(set([morph.parse(w)[0].normalized for w in words]), key = lambda w: w.normal_form)   

In [27]:
model = gensim.models.Word2Vec(sents, min_count=100)

In [18]:
dir(model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_do_train_job',
 '_load_specials',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_smart_save',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'cbow_mean',
 'clear_sims',
 'corpus_count',
 'create_binary_tree',
 'cum_table',
 'delete_temporary_training_data',
 'doesnt_match',
 'estimate_memory',
 'evaluate_word_pairs',
 'finalize_vocab',
 'hashfxn',
 'hs',
 'init_sims',
 'initialize_word_vectors',
 'intersect_word2vec_format',
 'iter',
 'layer1_size',
 'load',
 'load_word2vec_format',
 'log_accuracy',
 'log_evaluate_word_pairs',
 'make_cum_table',
 'max_vocab_size',


## Тут предложения приводим к нормальной форме с помощью texterra
Стало выкидывать ошибку 
HTTPError: 429 Client Error: Calm down for url: http://api.ispras.ru/texterra/v3.1/nlp/lemma?apikey=dd58e71c2c65fa4af14fb557f904ede7f3b30575&filtering=KEEPING&class=lemma


In [109]:
def progress(full, curr, checkpoint):
    if curr / full * 100 > checkpoint:
        print(curr / full * 100, end = '')
        print('% complited')
        return True
    
    else:
        return False
    
i = 0
h = 0.2
checkpoint = h
time = tm.clock()
for sent in sents:
    if progress(len(sents), i, checkpoint):
        time_per_checkpoint = tm.clock() - time
        print('Time per checkpoint:', end = ' ')
        print(time_per_checkpoint)
        print('Time per sentence:', end = ' ')
        time_per_sent = time_per_checkpoint / (len(sents) * h)
        print(time_per_sent)
        print('Estimated time:', end = ' ')
        print(time_per_checkpoint * (100 - checkpoint))
        time = tm.clock()
        checkpoint = checkpoint + h
        
    toks = tknzr.tokenize(sent)
    toks = [t.lower() for t in toks if t.isalnum()]
    sentence = ''
    for t in toks:
        sentence = sentence + t + ' '
    
    lemma_raw = isp_api.lemmatizationAnnotate(text = sentence)
    lemmatized = ''
    if lemma_raw.get('annotations').get('lemma') is not None:
        for l in lemma_raw.get('annotations').get('lemma'):
            lemmatized = lemmatized + l.get('value') + ' '
        
        sents[i] = lemmatized
    
    else:
        sents[i] = ''
        
    i = i + 1
    tm.sleep(0.1)
    
sents[0]

HTTPError: 429 Client Error: Calm down for url: http://api.ispras.ru/texterra/v3.1/nlp/lemma?apikey=dd58e71c2c65fa4af14fb557f904ede7f3b30575&filtering=KEEPING&class=lemma

In [108]:
aaa = 'В новом выпуске Инсайдов: Oukitel K6000 Plus получает крупное программное обновление и новый цвет корпуса; характеристики ZTE Nubia Z17 Lite обнаружены в Geekbench'
sample = isp_api.lemmatizationAnnotate(text = aaa)
print(type(sample))
bbb = ''
for s in sample.get('annotations').get('lemma'):
    bbb = bbb + s.get('value') + ' '
   
if sample.get('annotations').get('lemm') is not None:
    print("Azaz")

HTTPError: 429 Client Error: Calm down for url: http://api.ispras.ru/texterra/v3.1/nlp/lemma?apikey=dd58e71c2c65fa4af14fb557f904ede7f3b30575&filtering=KEEPING&class=lemma